In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(
# dataML.drop(['town', 'remaining_lease'], axis=1), # predictors
# dataML['town'], # target
# test_size=0.3, # percentage of samples in test set
# random_state=0)

Pseudocode of random forest with reference to HDB price dataset  

Note: data with high cardinality means each town (i.e. feature) is very unique

In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn import preprocessing
# #---- Random Forest
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import roc_auc_score
# X_train, X_test, y_train, y_test = train_test_split(
# dataML.drop(['town', 'remaining_lease'], axis=1), # predictors
# dataML['town'], # target
# test_size=0.3, # percentage of samples in test set
# random_state=0) # seed to ensure reproducibility
# # sanity check:
# if (1):
# # Let's find out labels present only in the training set
# unique_to_train_set = [
# y for y in y_train.unique() if y not in y_test.unique()
# ]
# print('len(unique_to_train_set)-----------------')
# print(len(unique_to_train_set))
# # Let's find out labels present only in the test set
# unique_to_test_set = [
# y for y in y_test.unique() if y not in y_train.unique()
# ]
# print('len(unique_to_test_set)-----------------')
# print(len(unique_to_test_set))

# # Random Forest model built on data with high cardinality for town
# # call the model
# rf = RandomForestClassifier(n_estimators=200, random_state=39)
# # train the model
# rf.fit(X_train[['floor_area_sqm', 'remaining_years', 'resale_price']], y_train)
# # make predictions on train and test set
# pred_train = rf.predict_proba(X_train[['floor_area_sqm', 'remaining_years', 'resale_price']])
# pred_test = rf.predict_proba(X_test[['floor_area_sqm', 'remaining_years', 'resale_price']].fillna(0))
# print('Training set results')
# print('Random Forests roc-auc: {}'.format(roc_auc_score(y_train, pred_train, multi_class='ovr')))
# print('Test set results')
# print('Random Forests roc-auc: {}'.format(roc_auc_score(y_test, pred_test, multi_class='ovr')))